<a href="https://colab.research.google.com/github/dramamine/ml-style-transfer-experimentation/blob/main/Marten_Style_Transfer_HD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stylize Your Images In 5 Minutes

Demo developed by [Sayak Paul](https://twitter.com/RisingSayak) and modified by [Marten Silbiger](https://metal-heart.org)

and adamp

Neural style transfer is one of the most interesting applications of deep learning. We've created a demo to help you recreate your images in the style of famous artists.

![](https://storage.googleapis.com/download.tensorflow.org/models/tflite/arbitrary_style_transfer/table.png)

The image and code id adapted from [this tutorial](https://www.tensorflow.org/lite/models/style_transfer/overview). 

## To start the demo click the play button to the left of each cell, one by one.

You need to run the following cells in order to get stylized images.

The easiest way to do that is to click on the cell and press Shift + Enter or press the play button. Each cell takes between 10-30 seconds to execute.

# Create an HD Render

In [ ]:
#@title 1. Setup 🧰
#@markdown Just run this cell as is. ***Don't modify the code block.*** The setup should not take more than two minutes. 
print("Installing tf-nightly...")
!pip uninstall -q -y tensorflow 
!pip install -q tf-nightly

from IPython.display import clear_output
import matplotlib.pyplot as plt
import tensorflow as tf
 
clear_output()

print("tf-nightly installed.")

print("Mounting google drive...")
#@title 2. Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')



# if you need to see inside or check the directory structure, you can use this
#!apt-get install tree
#!tree '/content/drive/MyDrive/images/'
print("google drive mounted.")

# Download the style bottleneck and transfer networks
print('Downloading the model files...')

style_predict_path = tf.keras.utils.get_file('style_predict.tflite', 'https://tfhub.dev/sayakpaul/lite-model/arbitrary-image-stylization-inceptionv3/int8/predict/1?lite-format=tflite')
style_transform_path = style_transform_path = tf.keras.utils.get_file('style_transform.tflite', 'https://tfhub.dev/sayakpaul/lite-model/arbitrary-image-stylization-inceptionv3/int8/transfer/1?lite-format=tflite')

print('Model files downloaded...')

# other deps
from IPython.display import display
import ipywidgets as widgets

import numpy as np
import io
import os

print('You are all set!')

In [ ]:
#@title 2. Configure 🍹
from PIL import Image

google_drive_base = '/content/drive/MyDrive/images/'
content_image_path = 'prettyearth/1568.jpg' #@param
style_image_path = 'alex-grey/wp2705517-alex-grey-backgrounds-desktop.jpg' #@param

#@title 3b. Divide our style image and content image into sub-blocks.
#@markdown Here you need to set the number of rows and columns for subdivisions.
#@markdown Each region should be 256x256 or greater, ex. 1920x1080 should be
#@markdown _at most_ 7 columns by 4 rows. Each cell was taking me about 90 seconds to generate.

cols = 3 #@param {type:"slider", min:1, max:7, step:1}
rows = 2 #@param {type:"slider", min:1, max:4, step:1}

#@markdown Should we calculate one style for the entire style image? Or should we 
#@markdown divide the style image into tiles just like we do with the the content?
use_tiled_style_image = False #@param {type:"boolean"}

#@markdown Edge size in final image. A corner piece will have (384-edge_size)^2
#@markdown of non-discarded content, while a center piece will be (384-2*edge_size)^2.
edge_size = 4 #@param {type:"slider", min:0, max:32, step:1}

content_image = Image.open(google_drive_base+content_image_path)
style_image = Image.open(google_drive_base+style_image_path)

print('files loaded.')

import math
def imshow(image, title=None):
  plt.imshow(image)
  if title:
    plt.title(title)
def get_array_of_pieces(title, img, cols, rows, show=False):
  pieces = []
  print(title,"image size:", img.size)
  w = math.floor(img.size[0]/cols)
  h = math.floor(img.size[1]/rows)
  print("using cell size:", w, "x", h)

  if (w < 256 or h < 256):
    print("WARNING: That size seems a bit small and will probably result in stretching.")

  # scale edge_size
  we = w*edge_size/384
  he = h*edge_size/384

  for r in range(0, rows):
    for c in range(0, cols):
      el = 0 if c == 0 else we
      er = 0 if c == cols-1 else we
      eu = 0 if r == 0 else he
      ed = 0 if r == rows-1 else he

      region = img.crop( (c*w-el, r*h-eu, (c+1)*w+er, (r+1)*h+ed) )
      pieces.append(region)
      if show:
        plt.subplot(rows, cols, r*cols+c+1)
        imshow(region)
  return pieces

# plt.figure(figsize=(10, 10))
content_pieces = get_array_of_pieces("content", content_image, cols, rows, True)
if use_tiled_style_image:
  style_pieces = get_array_of_pieces("style", style_image, cols, rows)
else: 
  style_pieces = list([style_image])


In [ ]:
#@title 3. Preprocess 👾

#@markdown ***Don't modify this code block.***

# Function to load an image from a file, and add a batch dimension.
def load_img(path_to_img):
  img = tf.io.read_file(path_to_img)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = img[tf.newaxis, :]
  return img

# Function to load an image from a file, and add a batch dimension.
def load_content_img(image_pixels):
    if image_pixels.shape[-1] == 4:
        image_pixels = Image.fromarray(image_pixels)
        img = image_pixels.convert('RGB')
        img = np.array(img)
        img = tf.convert_to_tensor(img)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = img[tf.newaxis, :]
        return img
    elif image_pixels.shape[-1] == 3:
        img = tf.convert_to_tensor(image_pixels)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = img[tf.newaxis, :]
        return img
    elif image_pixels.shape[-1] == 1:
        raise Error('Grayscale images not supported! Please try with RGB or RGBA images.')
    print('Exception not thrown')

# Function to pre-process by resizing an central cropping it.
def preprocess_image(image, target_dim):
  # Resize the image so that the shorter dimension becomes 256px.
  shape = tf.cast(tf.shape(image)[1:-1], tf.float32)
  short_dim = min(shape)
  scale = target_dim / short_dim
  new_shape = tf.cast(shape * scale, tf.int32)
  image = tf.image.resize(image, new_shape)

  # Central crop the image.
  image = tf.image.resize_with_crop_or_pad(image, target_dim, target_dim)

  return image

# Function to run style prediction on preprocessed style image.
def run_style_predict(preprocessed_style_image):
  # Load the model.
  interpreter = tf.lite.Interpreter(model_path=style_predict_path)

  # Set model input.
  interpreter.allocate_tensors()
  input_details = interpreter.get_input_details()
  interpreter.set_tensor(input_details[0]["index"], preprocessed_style_image)

  # Calculate style bottleneck.
  interpreter.invoke()
  style_bottleneck = interpreter.tensor(
      interpreter.get_output_details()[0]["index"]
      )()
  # print('Style Bottleneck Shape:', style_bottleneck.shape)
  return style_bottleneck

def preprocessor(img, res):
  img = load_content_img(np.array( img ))
  return preprocess_image(img, res)

print("Preprocessing content...")
tf_content_pieces = list(map(lambda x: 
    load_content_img(np.array(x)), content_pieces ))
preprocessed_content_pieces = list(map(lambda x: 
    preprocessor(x, 384), content_pieces ))
preprocessed_style_pieces = list(map(lambda x: 
    preprocessor(x, 256), style_pieces ))

print("Calculating bottlenecks...this may take awhile.")
style_bottlenecks = list(map(lambda x: 
    run_style_predict(x), preprocessed_style_pieces ))
style_bottleneck_contents = list(map(lambda x: 
    run_style_predict( preprocess_image(x, 256)), tf_content_pieces ))
print("Content length:", len(tf_content_pieces), 
      len(preprocessed_content_pieces), len(style_bottleneck_contents))
print("Style length:", len(preprocessed_style_pieces), 
      len(style_bottlenecks))

print("Finished preprocessing.")



In [ ]:
#@title 6. Stylize image 🥁

content_blending_ratio = 0.5 #@param {type:"slider", min:0, max:1, step:0.1}
#@markdown You're encouraged to play with the different values of `content_blending_ratio`.

# Run style transform on preprocessed style image
def run_style_transform(style_bottleneck, preprocessed_content_image):
  # Load the model.
  interpreter = tf.lite.Interpreter(model_path=style_transform_path)

  # Set model input.
  input_details = interpreter.get_input_details()
  interpreter.allocate_tensors()

  # Set model inputs.
  for index in range(len(input_details)):
    if input_details[index]["name"]=='Conv/BiasAdd':
      interpreter.set_tensor(input_details[index]["index"], style_bottleneck)
    elif input_details[index]["name"]=='content_image':
      interpreter.set_tensor(input_details[index]["index"], preprocessed_content_image)
  interpreter.invoke()

  # Transform content image.
  stylized_image = interpreter.tensor(
      interpreter.get_output_details()[0]["index"]
      )()

  return stylized_image

def stylize(preprocessed_content_image, style_bottleneck, style_bottleneck_content):
  # Blend the style bottleneck of style image and content image
  style_bottleneck_blended = content_blending_ratio * style_bottleneck_content \
                            + (1 - content_blending_ratio) * style_bottleneck

  # Stylize the content image using the style bottleneck.
  stylized_image = run_style_transform(style_bottleneck_blended, preprocessed_content_image)

  print('🐌', end="")
  return stylized_image

print("Processing", len(preprocessed_content_pieces), "cells of content...", end="")

if use_tiled_style_image:
  stylized_pieces = list(map(lambda x,y,z: stylize(x,y,z), preprocessed_content_pieces, style_bottlenecks, style_bottleneck_contents ))
else:
  stylized_pieces = list(map(lambda x,z: stylize(x,style_bottlenecks[0],z), preprocessed_content_pieces, style_bottleneck_contents ))

In [ ]:
#@title Sew it together and preview 🔮

plt.figure(figsize=(10, 10))

# @TODO parameterize
rows = 2
cols = 3

wh = 384

im = Image.new('RGB', (
    cols*wh-2*edge_size*(cols-1), 
    rows*wh-2*edge_size*(rows-1)
))


for r in range(0,rows):
  for c in range(0,cols):
    squeezed = tf.squeeze( stylized_pieces[c+r*cols] )
    imaged = tf.keras.preprocessing.image.array_to_img(squeezed)

    el = 0 if c == 0 else edge_size
    er = 0 if c == cols-1 else edge_size
    eu = 0 if r == 0 else edge_size
    ed = 0 if r == rows-1 else edge_size


    print("imaged size:")
    print(imaged.size)
    cropped = imaged.crop((el,eu,wh-er,wh-ed))
    print("cropped size:")
    print(cropped.size)

    im.paste(cropped, (
        c*wh - max(0, (2*c-1)*el), # left
        r*wh - max(0, (2*r-1)*eu) # up
    ))

plt.subplot(1,1,1)
imshow(im)

In [ ]:
#@title Save your results 💥
#@markdown If you experiment with the different `content_blending_ratio` values make sure you run this code block again in order to store your results online.
import re

def get_nice_name(path):
  return re.search(r'.*/(.*).jpg', path).group(1)

output = "{0}output/{1}-{2}-hd-fusion-{3}x{4}-blend{5}-edge{6}{7}.jpg".format(
    google_drive_base,
    get_nice_name(content_image_path),
    get_nice_name(style_image_path),
    im.size[0],
    im.size[1],
    int(10*content_blending_ratio),
    edge_size,
    "-tiled" if use_tiled_style_image else ""
)

im.save(output, "JPEG")
print("Saved to:", output)